In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
import time
import concurrent.futures
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import mplfinance as mpf
import plotly.graph_objects as go
import nbformat
import plotly.io as pio
from sklearn.preprocessing import StandardScaler
from TradingModelTrainer import TradingModelTrainer
from DataProvider import DataProvider
from GeneticAlgorithm import GeneticAlgorithm



In [2]:
def save_models(models: list[Sequential], folder: str = "Models"):
    """Save models to a specified folder."""
    if not os.path.exists(folder):
        os.makedirs(folder)
    for i, model in enumerate(models):
        model_path = os.path.join(folder, f"model_{i + 1}.h5")
        model.save(model_path)
        print(f"Model {i + 1} saved at {model_path}")

def load_models(model_names: list[str], folder: str = "Models") -> list[Sequential]:
    """Load models from a specified folder."""
    loaded_models = []
    for model_name in model_names:
        model_path = os.path.join(folder, model_name)
        if os.path.exists(model_path):
            model = load_model(model_path)
            loaded_models.append(model)
            print(f"Loaded model from {model_path}")
        else:
            print(f"Model file {model_path} not found.")
    return loaded_models

In [3]:
data_provider = DataProvider('Data/Binance_BTCUSDT_2024_minute.csv')
raw_df = data_provider.get_raw_data()
normalized_df = data_provider.get_normalized_data()


In [4]:
TIMESTEPS = 100  # Adjust based on your data's characteristics
FEATURES = 7    # As determined from your data
POPULATION_SIZE = 10  # Number of models in each generation
GENERATIONS = 10  # Number of generations for evolving models

# Initialize the TradingModelTrainer
trainer = TradingModelTrainer(trainData=normalized_df,  # Replace with your training data DataFrame
                              tradeData=raw_df,  # Replace with your trading data DataFrame
                              initial_capital=10000,   # Example initial capital
                              trade_size_dollars=5000) # Example trade size

ga = GeneticAlgorithm(trainer)

In [5]:
population = []#load_models(["model_1.h5", "model_2.h5", "model_3.h5", "model_4.h5", "model_5.h5"])

In [6]:
for _ in range(POPULATION_SIZE):
    model = trainer.create_lstm_model(TIMESTEPS, FEATURES)
    population.append(model)

final_population = ga.genetic_algorithm(population, GENERATIONS, TIMESTEPS, FEATURES)



gen: 0 / 10
Generation 1 complete. Best Sharpe Ratio: -8.447751016690168
gen: 1 / 10
Generation 2 complete. Best Sharpe Ratio: -8.447751016690168
gen: 2 / 10
Generation 3 complete. Best Sharpe Ratio: -8.447751016690168
gen: 3 / 10
Generation 4 complete. Best Sharpe Ratio: -3.8894527078328496
gen: 4 / 10
Generation 5 complete. Best Sharpe Ratio: -3.8894527078328496
gen: 5 / 10
Generation 6 complete. Best Sharpe Ratio: -3.8894527078328496
gen: 6 / 10
Generation 7 complete. Best Sharpe Ratio: -3.6902604796583676
gen: 7 / 10
Generation 8 complete. Best Sharpe Ratio: -3.6902604796583676
gen: 8 / 10
Generation 9 complete. Best Sharpe Ratio: -3.6902604796583676
gen: 9 / 10
Generation 10 complete. Best Sharpe Ratio: -3.6902604796583676


In [7]:
learning_df = trainer.get_learning_history_df()
learning_df

,final_capital,sharpe_ratio
0,10306.238640,-8.447751
1,10306.238640,-8.447751
2,10306.238640,-8.447751
3,9408.150638,-3.889453
4,9408.150638,-3.889453
5,9408.150638,-3.889453
6,9216.921971,-3.690260
7,9216.921971,-3.690260
8,9216.921971,-3.690260
9,9216.921971,-3.690260


In [8]:
traders = []

for model in final_population:
    trader = trainer.simulate_trading(model, TIMESTEPS)
    traders.append(trader)

In [20]:
trader = traders[0]
result = trader.get_trade_history_df()
# final_population now contains the evolved models
result

,index,entry_price,exit_price,type,contracts,profit_loss,close_index
0,45,42465.52,44191.11,short,0.117743,-203.175423,9908.0
1,9908,44191.11,44192.05,long,0.113145,0.106356,9910.0
2,9910,44192.05,40941.11,short,0.113143,367.819551,25016.0
3,25016,40941.11,40895.24,long,0.122127,-5.601949,25177.0
4,25177,40895.24,39036.00,short,0.122264,227.317409,26547.0
5,26547,39036.00,38953.29,long,0.128087,-10.594067,26553.0
6,26553,38953.29,38940.12,short,0.128359,1.690486,26559.0
7,26559,38940.12,38966.01,long,0.128402,3.324335,26624.0
8,26624,38966.01,38973.58,short,0.128317,-0.971359,26735.0
9,26735,38973.58,38864.16,long,0.128292,-14.037715,26740.0


In [21]:
result.describe()

,index,entry_price,exit_price,contracts,profit_loss,close_index
count,29.000000,29.000000,28.000000,29.000000,28.000000,28.000000
mean,30726.344828,41773.703448,41748.995714,0.120598,-27.967072,31822.107143
std,14429.865966,3851.928319,3920.270901,0.010178,215.593829,13409.814687
min,45.000000,38622.940000,38622.940000,0.101221,-1015.282322,9908.000000
25%,26559.000000,38966.010000,38962.830000,0.113145,-10.952785,26607.750000
50%,26791.000000,40007.810000,39964.105000,0.124976,-1.013105,26816.000000
75%,28150.000000,44191.110000,44191.345000,0.128317,4.321478,28186.500000
max,57362.000000,49396.690000,49396.690000,0.129457,367.819551,57362.000000


In [22]:

trader.calculate_sharpe_ratio()

-3.6902604796583676

In [23]:
trader.current_capital

9216.921970502077

In [13]:
# # Sample data for demonstration
# import pandas as pd

# from Trader import Trader


# data = {
#     'close': [100, 105, 110, 105, 100, 95, 90, 95, 100, 80]  # Simulated close prices
# }
# df = pd.DataFrame(data)

# # Initialize the Trader
# trade_size_dollars = 1000  # Trade size in dollars
# initial_capital = 10000    # Initial capital in dollars
# trader = Trader(df, trade_size_dollars, initial_capital)

# # Define a series of actions: 0 for long, 1 for wait, 2 for short
# actions = [0, 1, 2, 1, 1, 1, 0, 1, 1, 2]  # Example sequence of actions

# # Simulate trading
# trader.trade(actions)

# # Get the trade history
# trade_history_df = trader.get_trade_history_df()
# trade_history_df

In [14]:
#save_models(final_population)